In [3]:
import pandas as pd
import datetime as dt
import numpy as np
from tqdm import tqdm_notebook
import collections
tqdm_notebook().pandas()
from utils import *
from sklearn.cluster import KMeans
from sklearn.model_selection import GroupKFold, GroupShuffleSplit
import lightgbm as lgb

A Jupyter Widget

## Считываем и подготавливаем данные

In [5]:
train_data = pd.read_csv("train_set.csv").rename(columns={"pos_adress_lat": "pos_address_lat",
                                                                                "pos_adress_lon": "pos_address_lon"})
test_data = pd.read_csv("test_set.csv")

/Users/katerina198b/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/Users/katerina198b/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [39]:
l1 = pd.read_csv("train_set.csv")
l1.head()

/Users/katerina198b/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,11) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


AttributeError: 'DataFrame' object has no attribute 'cols'

In [42]:
l1.columns

Index(['amount', 'atm_address', 'atm_address_lat', 'atm_address_lon', 'city',
       'country', 'currency', 'customer_id', 'home_add_lat', 'home_add_lon',
       'mcc', 'pos_address', 'pos_adress_lat', 'pos_adress_lon', 'terminal_id',
       'transaction_date', 'work_add_lat', 'work_add_lon'],
      dtype='object')

In [43]:
l1.head()

,amount,atm_address,atm_address_lat,atm_address_lon,city,country,currency,customer_id,home_add_lat,home_add_lon,mcc,pos_address,pos_adress_lat,pos_adress_lon,terminal_id,transaction_date,work_add_lat,work_add_lon
0,2.884034,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,11606fde0c814ce78e0d726e39a0a5ee,2017-07-15,59.847,30.177
1,2.775633,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,e9647a5e1eacfb06713b6af755ccc595,2017-10-27,59.847,30.177
2,3.708368,NaN,NaN,NaN,St Petersburg,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5992,"PR.MARSHALA ZHUKOVA,31St Petersburg190000 7...",59.858198,30.229024,df06c1fcd3718a514535ae822785f716,2017-10-03,59.847,30.177
3,2.787498,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,6c5e5793ebc984fb72875feffff62854,2017-09-09,59.847,30.177
4,2.892510,NaN,NaN,NaN,ST PETERSBURG,RUS,643.0,0dc0137d280a2a82d2dc89282450ff1b,59.851,30.232,5261,NaN,59.844072,30.179153,0576445d74e374c92c0902e612fca356,2017-07-06,59.847,30.177


In [6]:
train = preprocess(train_data)

/Users/katerina198b/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3110: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [7]:
train.pos_address_lat.fillna(train.atm_address_lat,inplace=True)
train.pos_address_lon.fillna(train.atm_address_lon,inplace=True)

In [8]:
train.drop(['atm_address_lat', 'atm_address_lon', 'atm_address', 'atm_address_lat_upd', 'atm_address_lon_upd', 'terminal_id_upd'], axis=1, inplace=True)

In [9]:
train_data.shape, train.shape

((1224734, 18), (1125751, 15))

## Добавляем день недели

In [10]:
train.transaction_date = train.transaction_date.progress_apply(lambda x: parse(x))

A Jupyter Widget

In [11]:
train["week_day"] = train.transaction_date.apply(lambda x: x.weekday())

## Добавляем количество терминалов рядом, которыми пользовался пользователь

In [12]:
train_df = number_near_terminals(train.copy())

A Jupyter Widget

## Минимальное максимальное и среднее расстояние от данного терминала до других терминалов, которыми пользовался пользователь

In [15]:
import utils
import importlib
utils = importlib.reload(utils)
from utils import *

In [16]:
train_df = dist_features(train_df.copy())

A Jupyter Widget

A Jupyter Widget

A Jupyter Widget

In [17]:
train_df.to_csv('prep_train.csv', index=False)

In [18]:
train_df = pd.read_csv('prep_train.csv')


train_df.transaction_date = train_df.transaction_date.progress_apply(lambda x: parse(x))

train_df["week_day"] = train_df.transaction_date.apply(lambda x: x.weekday())



/Users/katerina198b/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (8) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


A Jupyter Widget

## Сколько раз этот терминал был использован данным пользователем

In [19]:
## count_ters
train_df = ter_using(train_df)

## Была ли совершена транцакция рядом с домом или работой

In [20]:
train_df = get_target(train_df)

## Сколько раз вообще использовали это терминал

In [21]:
#train_df = pd.get_dummies(train_df, columns=["week_day"])
train_df = make_pairs(train_df)
train_df = number_equal_coordinates(train_df)

In [22]:
mapping = train_df.groupby("terminal_id").customer_id.size().to_dict()
train_df['ter_customers'] = train_df.terminal_id.map(mapping)

## Сколько раз пользовались в выходной день

In [23]:
def ter_using_w(df):
    ldf = df.copy()
    ldf = ldf.merge(ldf[ldf.week_day.isin([5,6])].groupby(["customer_id", 'terminal_id']).size().reset_index(name='count_ters_w'), how='left', left_on=['customer_id','terminal_id'], right_on=['customer_id','terminal_id']) 
    ldf.count_ters_w.fillna(0, inplace=True)
    return ldf

In [24]:
train_df = ter_using_w(train_df)

In [25]:
train_df.mcc = train_df.mcc.apply(lambda x: int(str(x).replace(",","")))

## Среднее значение по каждой координате терминала для пользователя

In [26]:
train_df = train_df.merge(train_df.groupby('customer_id').pos_address_lat.median().reset_index(name='pos_lat_median'), how='left',\
                        on='customer_id')
train_df = train_df.merge(train_df.groupby('customer_id').pos_address_lon.median().reset_index(name='pos_lon_median'), how='left',\
                        on='customer_id')


## Разбиение на train test

In [27]:
## разбиение train и test
folds_generator = GroupShuffleSplit(1,random_state=42)
ind = folds_generator.split(train_df, train_df.y_home, groups=train_df.customer_id)
folds = []
for el, er in ind:
    folds += [(el, er)]

In [28]:
temp = train_df.iloc[folds[0][0]].copy()
temp_test = train_df.iloc[folds[0][1], :].copy()

HOME_GM = train_df.y_home.mean()
WORK_GM = train_df.y_work.mean()

In [29]:
import utils
import importlib
utils = importlib.reload(utils)
from utils import *

In [30]:
temp = mean_enc_train(temp.copy(), "count_ters", typ='home', globalmean=HOME_GM)
temp = mean_enc_train(temp.copy(), "mcc", typ='home', globalmean=HOME_GM)
temp = mean_enc_train(temp.copy(), "num_near_terminals", typ='home', globalmean=HOME_GM)
temp = mean_enc_train(temp.copy(), "week_day", typ='home', globalmean=HOME_GM)

/Users/katerina198b/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [31]:
temp = mean_enc_train(temp.copy(), "count_ters", typ='work', globalmean=WORK_GM)
temp = mean_enc_train(temp.copy(), "mcc", typ='work', globalmean=WORK_GM)
temp = mean_enc_train(temp.copy(), "num_near_terminals", typ='work', globalmean=WORK_GM)
temp = mean_enc_train(temp.copy(), "week_day", typ='work', globalmean=WORK_GM)

/Users/katerina198b/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:179: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [32]:
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "count_ters", typ='home', globalmean=HOME_GM)
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "mcc", typ='home', globalmean=HOME_GM)
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "num_near_terminals", typ='home', globalmean=HOME_GM)
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "week_day", typ='home', globalmean=HOME_GM)

In [33]:
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "count_ters", typ='work', globalmean=WORK_GM)
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "mcc", typ='work', globalmean=WORK_GM)
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "num_near_terminals", typ='work', globalmean=WORK_GM)
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "week_day", typ='work', globalmean=WORK_GM)

In [34]:
cols = set(list(temp.columns)) - set(['ter_customers','dist_to_mean2',\
        'dist_to_min2','dist_to_max2', 'merchants',"number_of_equal_mean",'week_day_0', 'week_day_1',
       'week_day_2', 'week_day_3', 'week_day_4', 'week_day_5', 'week_day_6',\
                                      'terminal_id_mean', 'city_mean',\
       'cust_id_upd', 'city_upd', 'currency_upd', 'country_upd',\
       'mean_target_cutomer', 'mean_target_terminal', 'mean_target_city',\
       'mean_target_country',  'near_ter', 'coord_prob', 'lon_prob', 'lat_prob',\
       'amount_mean', 'pos', 'pos_address', 'atm_address',\
       'atm_address_lat', 'atm_address_lon', 'city', 'country', 'currency',\
       'customer_id', 'home_add_lat', 'home_add_lon','terminal_id',\
       'transaction_date', 'work_add_lat', 'work_add_lon', 'y_home', 'y_work',\
    'mcc_mean_work', 'count_ters_mean_work', 'week_day_mean_work', 'num_near_terminals_mean_work'])
cols = list(cols)

In [146]:
cols

['count_ters',
 'pos_address_lat',
 'dist_to_min',
 'num_near_terminals_mean_home',
 'num_near_terminals',
 'dist_to_max',
 'mcc',
 'number_of_equal',
 'week_day_mean_home',
 'count_ters_w',
 'dist_to_mean',
 'week_day',
 'count_ters_mean_home',
 'pos_lat_median',
 'pos_lon_median',
 'mcc_mean_home',
 'pos_address_lon',
 'amount']

In [35]:
model_home = fit(temp, temp.y_home, cols, typ="home", mt='lgb')
model_work = fit(temp, temp.y_work, cols, typ="work", mt='lgb')

In [36]:
preds = predict(model_home, temp_test, cols, mt='lgb')
evaluate(train_df.copy(), preds, "home")

/Users/katerina198b/Documents/maсhine_learning/utils.py:370: RuntimeWarning: invalid value encountered in less
  r = np.sum(np.sqrt(res[:, 0]**2+res[:, 1]**2) < .02)/res.shape[0]


0.42799999999999999

In [32]:
preds = predict(model_home, temp_test, cols, mt='lgb')
evaluate(train_df.copy(), preds, "home")

/Users/antonzaharenkov/raifaizen/code/utils.py:306: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  res = np.array(df.loc[preds.index]) - np.array(preds)
/Users/antonzaharenkov/raifaizen/code/utils.py:307: RuntimeWarning: invalid value encountered in less
  r = np.sum(np.sqrt(res[:, 0]**2+res[:, 1]**2) < .02)/res.shape[0]


0.42649999999999999

In [37]:
preds = predict(model_work, temp_test, cols, mt='lgb')
evaluate(train_df.copy(), preds, "work")

/Users/katerina198b/Documents/maсhine_learning/utils.py:370: RuntimeWarning: invalid value encountered in less
  r = np.sum(np.sqrt(res[:, 0]**2+res[:, 1]**2) < .02)/res.shape[0]


0.154

In [94]:
(0.4265+0.151)/2

0.28875

In [182]:
preds = predict(model_home, temp_test, cols, mt='lgb')
evaluate(train_df.copy(), preds, "work")

/Users/antonzaharenkov/raifaizen/code/utils.py:306: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  res = np.array(df.loc[preds.index]) - np.array(preds)
/Users/antonzaharenkov/raifaizen/code/utils.py:307: RuntimeWarning: invalid value encountered in less
  r = np.sum(np.sqrt(res[:, 0]**2+res[:, 1]**2) < .02)/res.shape[0]


0.063

In [102]:
def fit(params, X_train, y_train, temp_test, cols, num_boost_round=101, mt='lgb', typ="home"):
    if typ=="home":
        good_merch = X_train.home_add_lat.dropna().index
        X_train = X_train.loc[good_merch]
        y_train = y_train.loc[good_merch]
    else:
        good_merch = X_train.work_add_lat.dropna().index
        X_train = X_train.loc[good_merch]
        y_train = y_train.loc[good_merch]
    if mt == 'lgb':
        lgb_train = lgb.Dataset(X_train[cols], y_train)

    lgb_train = lgb.Dataset(X_train[cols], y_train)
    lgb_eval = lgb.Dataset(temp_test[cols], temp_test.y_home, reference=lgb_train)
        # specify your configurations as a dict
#         params = {
#             'boosting_type': 'gbdt',
#             'objective': 'binary',
#             'metric': 'binary_logloss',
#             'bagging_freq': 1,
#             'colsample_bytree': '0.514',
#             'learning_rate': '0.013',
#             'feature_fraction': '0.625',
#             'bagging_fraction': '0.844',
#             'num_leaves': 30,
#             'verbose': 1
#             #'is_unbalance':True
#         }

    model = lgb.train(params,
                lgb_train,
                valid_sets=lgb_eval,
                  verbose_eval=0,
                  early_stopping_rounds=30,
                num_boost_round=500)

    return model


def objective(params):
    params = {
        'boosting_type': 'gbdt',  
        'objective': 'binary',
        'metric': 'binary_logloss',
        'bagging_freq': int(params['bagging_freq']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'learning_rate': '{:.3f}'.format(params['learning_rate']),
        'feature_fraction': '{:.3f}'.format(params['feature_fraction']),
        'bagging_fraction': '{:.3f}'.format(params['bagging_fraction']),
        'num_leaves': int(params['num_leaves']),
        'is_unbalance':True
    }
    model_home = fit(params, temp, temp.y_home, temp_test, cols, typ="home", mt='lgb')
    model_work = fit(params, temp, temp.y_work, temp_test, cols, typ="work", mt='lgb')

    
    preds = predict(model_home, temp_test, cols, mt='lgb')
    score1 = evaluate(train_df.copy(), preds, "home")
    
    preds = predict(model_work, temp_test, cols, mt='lgb')
    score2 = evaluate(train_df.copy(), preds, "work")
    print("Score full: {:.3f}, home: {:.3f}, work: {:.3f}, params {}".format((score1+score2)/2, score1, score2, params))
    return 1-(score1+score2)/2

space = {
    'num_leaves': hp.quniform('num_leaves', 8, 128, 2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'bagging_freq': hp.quniform('bagging_freq', 0, 30, 3),
    'learning_rate': hp.uniform('learning_rate', 0.005, 0.5),
    'feature_fraction': hp.uniform('feature_fraction', 0.6, 0.95),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.6, 0.95)}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=1000)

/Users/antonzaharenkov/raifaizen/code/utils.py:307: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  res = np.array(df.loc[preds.index]) - np.array(preds)
/Users/antonzaharenkov/raifaizen/code/utils.py:308: RuntimeWarning: invalid value encountered in less
  r = np.sum(np.sqrt(res[:, 0]**2+res[:, 1]**2) < .02)/res.shape[0]


Score full: 0.275, home: 0.420, work: 0.131, params {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss', 'bagging_freq': 0, 'colsample_bytree': '0.652', 'learning_rate': '0.293', 'feature_fraction': '0.897', 'bagging_fraction': '0.634', 'num_leaves': 60, 'is_unbalance': True, 'verbose': 1}
Score full: 0.275, home: 0.422, work: 0.129, params {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss', 'bagging_freq': 0, 'colsample_bytree': '0.376', 'learning_rate': '0.282', 'feature_fraction': '0.604', 'bagging_fraction': '0.791', 'num_leaves': 26, 'is_unbalance': True, 'verbose': 1}
Score full: 0.280, home: 0.420, work: 0.140, params {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss', 'bagging_freq': 15, 'colsample_bytree': '0.451', 'learning_rate': '0.007', 'feature_fraction': '0.674', 'bagging_fraction': '0.840', 'num_leaves': 26, 'is_unbalance': True, 'verbose': 1}
Score full: 0.284, home: 0.430, work: 0.139, params {'

Score full: 0.281, home: 0.426, work: 0.136, params {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss', 'bagging_freq': 3, 'colsample_bytree': '0.482', 'learning_rate': '0.141', 'feature_fraction': '0.646', 'bagging_fraction': '0.751', 'num_leaves': 18, 'is_unbalance': True, 'verbose': 1}
Score full: 0.285, home: 0.425, work: 0.144, params {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss', 'bagging_freq': 6, 'colsample_bytree': '0.692', 'learning_rate': '0.008', 'feature_fraction': '0.702', 'bagging_fraction': '0.720', 'num_leaves': 48, 'is_unbalance': True, 'verbose': 1}
Score full: 0.279, home: 0.422, work: 0.136, params {'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss', 'bagging_freq': 9, 'colsample_bytree': '0.562', 'learning_rate': '0.224', 'feature_fraction': '0.671', 'bagging_fraction': '0.644', 'num_leaves': 8, 'is_unbalance': True, 'verbose': 1}
Score full: 0.287, home: 0.429, work: 0.144, params {'bo

KeyboardInterrupt: 

In [100]:
temp.columns

Index(['amount', 'city', 'country', 'currency', 'customer_id', 'home_add_lat',
       'home_add_lon', 'mcc', 'pos_address', 'pos_address_lat',
       'pos_address_lon', 'terminal_id', 'transaction_date', 'work_add_lat',
       'work_add_lon', 'week_day', 'num_near_terminals', 'dist_to_mean',
       'dist_to_min', 'dist_to_max', 'count_ters', 'y_home', 'y_work', 'pos',
       'number_of_equal', 'count_ters_w', 'count_ters_mean_home',
       'mcc_mean_home', 'num_near_terminals_mean_home', 'week_day_mean_home',
       'count_ters_mean_work', 'mcc_mean_work', 'num_near_terminals_mean_work',
       'week_day_mean_work'],
      dtype='object')

In [103]:
cols = set(list(temp.columns)) - set(['ter_customers','dist_to_mean2',\
        'dist_to_min2','dist_to_max2', 'merchants',"number_of_equal_mean", 'week_day','week_day_0', 'week_day_1',
       'week_day_2', 'week_day_3', 'week_day_4', 'week_day_5', 'week_day_6',\
                                      'terminal_id_mean', 'city_mean',\
       'cust_id_upd', 'city_upd', 'currency_upd', 'country_upd',\
       'mean_target_cutomer', 'mean_target_terminal', 'mean_target_city',\
       'mean_target_country',  'near_ter', 'coord_prob', 'lon_prob', 'lat_prob',\
       'amount_mean','week_day', 'pos', 'pos_address', 'atm_address',\
       'atm_address_lat', 'atm_address_lon', 'city', 'country', 'currency',\
       'customer_id', 'home_add_lat', 'home_add_lon','terminal_id',\
       'transaction_date', 'work_add_lat', 'work_add_lon', 'y_home', 'y_work',\
    'mcc_mean_work', 'count_ters_mean_work', 'week_day_mean_work', 'num_near_terminals_mean_work'])
cols = list(cols)

In [104]:
cols

['count_ters',
 'pos_address_lat',
 'dist_to_min',
 'num_near_terminals_mean_home',
 'num_near_terminals',
 'mcc',
 'number_of_equal',
 'count_ters_w',
 'dist_to_mean',
 'count_ters_mean_home',
 'week_day_mean_home',
 'amount',
 'mcc_mean_home',
 'pos_address_lon',
 'dist_to_max']

In [115]:
temp.columns

Index(['amount', 'city', 'country', 'currency', 'customer_id', 'home_add_lat',
       'home_add_lon', 'mcc', 'pos_address', 'pos_address_lat',
       'pos_address_lon', 'terminal_id', 'transaction_date', 'work_add_lat',
       'work_add_lon', 'week_day', 'num_near_terminals', 'dist_to_mean',
       'dist_to_min', 'dist_to_max', 'count_ters', 'y_home', 'y_work', 'pos',
       'number_of_equal', 'dist_to_mean2', 'dist_to_min2', 'dist_to_max2',
       'ter_customers', 'count_ters_mean', 'mcc_mean',
       'num_near_terminals_mean', 'week_day_mean'],
      dtype='object')

# SUBMIT

In [34]:
test_data = pd.read_csv("../input/test_set.csv.gz", compression="gzip")

/Users/antonzaharenkov/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (1,8,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [35]:
test = preprocess(test_data)

/Users/antonzaharenkov/anaconda3/lib/python3.6/site-packages/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [36]:
test.pos_address_lat.fillna(test.atm_address_lat,inplace=True)
test.pos_address_lon.fillna(test.atm_address_lon,inplace=True)

In [37]:
test.drop(['atm_address_lat', 'atm_address_lon', 'atm_address', 'atm_address_lat_upd', 'atm_address_lon_upd', 'terminal_id_upd'], axis=1, inplace=True)

In [38]:
test_data.shape, test.shape

((1265470, 14), (1165026, 11))

In [39]:
test.transaction_date = test.transaction_date.progress_apply(lambda x: parse(x))

A Jupyter Widget

In [40]:
test["week_day"] = test.transaction_date.apply(lambda x: x.weekday())

KeyboardInterrupt: 

In [ ]:
test_df = number_near_terminals(test.copy())

In [ ]:
test_df = utils.dist_features(test_df.copy())

In [ ]:
test_df.to_csv('prep_test.csv', index=False)

In [41]:
test_df = pd.read_csv('prep_test.csv')

/Users/antonzaharenkov/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2698: DtypeWarning: Columns (5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [42]:
test_df.transaction_date = test_df.transaction_date.progress_apply(lambda x: parse(x))

test_df["week_day"] = test_df.transaction_date.apply(lambda x: x.weekday())



A Jupyter Widget

In [43]:
test_df = ter_using(test_df)

In [44]:
#train_df = pd.get_dummies(train_df, columns=["week_day"])
test_df = make_pairs(test_df)
test_df = number_equal_coordinates(test_df)

In [45]:
test_df.mcc = test_df.mcc.apply(lambda x: int(str(x).replace(",","")))

In [155]:
test_df["week_day"] = test_df.transaction_date.apply(lambda x: x.weekday())

In [101]:
def ter_using_w(df):
    ldf = df.copy()
    ldf = ldf.merge(ldf[ldf.week_day.isin([5,6])].groupby(["customer_id", 'terminal_id']).size().reset_index(name='count_ters_w'), how='left', left_on=['customer_id','terminal_id'], right_on=['customer_id','terminal_id']) 
    ldf.count_ters_w.fillna(0, inplace=True)
    return ldf

In [46]:
test_df = ter_using_w(test_df)

In [47]:
set(train_df.columns) -  set(test_df.columns)

{'home_add_lat',
 'home_add_lon',
 'work_add_lat',
 'work_add_lon',
 'y_home',
 'y_work'}

In [48]:
temp = train_df.copy()
temp_test = test_df.copy()

HOME_GM = train_df.y_home.mean()
WORK_GM = train_df.y_work.mean()

In [49]:
temp = mean_enc_train(temp.copy(), "count_ters", typ='home', globalmean=HOME_GM)
temp = mean_enc_train(temp.copy(), "mcc", typ='home', globalmean=HOME_GM)
temp = mean_enc_train(temp.copy(), "num_near_terminals", typ='home', globalmean=HOME_GM)
temp = mean_enc_train(temp.copy(), "week_day", typ='home', globalmean=HOME_GM)

/Users/antonzaharenkov/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [50]:
temp = mean_enc_train(temp.copy(), "count_ters", typ='work', globalmean=WORK_GM)
temp = mean_enc_train(temp.copy(), "mcc", typ='work', globalmean=WORK_GM)
temp = mean_enc_train(temp.copy(), "num_near_terminals", typ='work', globalmean=WORK_GM)
temp = mean_enc_train(temp.copy(), "week_day", typ='work', globalmean=WORK_GM)

/Users/antonzaharenkov/anaconda3/lib/python3.6/site-packages/pandas/core/indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


In [51]:
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "count_ters", typ='home', globalmean=HOME_GM)
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "mcc", typ='home', globalmean=HOME_GM)
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "num_near_terminals", typ='home', globalmean=HOME_GM)
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "week_day", typ='home', globalmean=HOME_GM)

In [52]:
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "count_ters", typ='work', globalmean=WORK_GM)
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "mcc", typ='work', globalmean=WORK_GM)
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "num_near_terminals", typ='work', globalmean=WORK_GM)
temp_test = mean_enc_test(temp.copy(), temp_test.copy(), "week_day", typ='work', globalmean=WORK_GM)

In [53]:
import utils
import importlib
utils = importlib.reload(utils)
from utils import *

In [55]:
model_home = fit(temp, temp.y_home, cols, typ="home", mt='lgb')

In [56]:
model_work = fit(temp, temp.y_work, cols, typ="work", mt='lgb')

In [54]:
cols = set(list(temp.columns)) - set([ 'ter_customers','dist_to_mean2', 'dist_to_min2','dist_to_max2', 'merchants',"number_of_equal_mean", 'week_day','week_day_0', 'week_day_1',
       'week_day_2', 'week_day_3', 'week_day_4', 'week_day_5', 'week_day_6',\
                                      'terminal_id_mean', 'city_mean',\
       'cust_id_upd', 'city_upd', 'currency_upd', 'country_upd',\
       'mean_target_cutomer', 'mean_target_terminal', 'mean_target_city',\
       'mean_target_country',  'near_ter', 'coord_prob', 'lon_prob', 'lat_prob',\
       'amount_mean','week_day', 'pos', 'pos_address','mcc', 'atm_address',\
       'atm_address_lat', 'atm_address_lon', 'city', 'country', 'currency',\
       'customer_id', 'home_add_lat', 'home_add_lon','terminal_id',\
       'transaction_date', 'work_add_lat', 'work_add_lon', 'y_home', 'y_work'])
cols = list(cols)

In [148]:
cols

['count_ters',
 'week_day_mean_home',
 'count_ters_w',
 'pos_address_lon',
 'num_near_terminals',
 'dist_to_min',
 'num_near_terminals_mean_home',
 'dist_to_max',
 'amount',
 'count_ters_mean_home',
 'pos_address_lat',
 'dist_to_mean',
 'mcc_mean_home',
 'number_of_equal']

In [106]:
temp_test.shape, temp_test_df.shape

((226073, 32), (1165026, 24))

In [194]:
preds_work = preds_home.rename(columns={"lat":"_WORK_LAT_",
                           "lon":"WORK_LON_"})

In [196]:
preds_work

,_WORK_LAT_,WORK_LON_
customer_id,,
0001f322716470bf9bfc1708f06f00fc,44.743860,37.724392
0027a7618d97cc9fbda55fac457eaeb7,55.637990,37.222875
003fa58414cc55531fcc38423bea8f8e,55.761793,37.771986
005194bf7238734eb49c142258c5a263,51.712323,39.157982
005d6f1ee97276374cefe4d0c65e9a89,55.770000,37.594000
007525af0bc3ce72137b586a298f7b98,55.821298,37.370401
0086aa6c4ebc89dd5a20eed71a625df1,55.737744,37.399083
0089100aa33a523bd2e697dec264a71a,55.418701,37.831608
008b0844115d2c7aafe51ccb3c4b2c2e,55.660844,37.573024


In [57]:
preds_home0 = predict(model_home, temp_test, cols, mt='lgb')
preds_work0 = predict(model_work, temp_test, cols, mt='lgb')

In [58]:
preds_home = preds_home0.rename(columns={"lat":"_HOME_LAT_",
                           "lon":"HOME_LON_"})
preds_work = preds_work0.rename(columns={"lat":"_WORK_LAT_",
                           "lon":"WORK_LON_"})

In [59]:
preds2 = pd.merge(preds_work, preds_home, left_index=True, right_index=True)

In [60]:
preds2.shape

(9997, 4)

In [61]:
submit = pd.read_csv("../input/sample.csv")

In [62]:
submit

,_ID_,_HOME_LAT_,_HOME_LON_,_WORK_LAT_,_WORK_LON_
0,00021683ccb416637fe9a4cd35e4606e,0,0,0,0
1,0002d0f8a642272b41c292c12ab6e602,0,0,0,0
2,0004d182d9fede3ba2534b2d5e5ad27e,0,0,0,0
3,0008c2445518c9392cb356c5c3db3392,0,0,0,0
4,000b373cc4969c0be8e0933c08da67e1,0,0,0,0
5,000c589e94c95984721de4b2bfb9ee4e,0,0,0,0
6,001611e3ac051a0ec91c88bbd9dbeb5a,0,0,0,0
7,001691ae3885e80add35148a01e75206,0,0,0,0
8,002044159304738ea7e3598131809851,0,0,0,0
9,0026319faa345a573522f0a04f5c55bb,0,0,0,0


In [63]:
preds2 = preds2.loc[submit._ID_]

In [64]:
preds2.index.name="_ID_"

In [65]:
preds2

,_WORK_LAT_,WORK_LON_,_HOME_LAT_,HOME_LON_
_ID_,,,,
00021683ccb416637fe9a4cd35e4606e,55.023355,82.914724,55.038211,82.977366
0002d0f8a642272b41c292c12ab6e602,44.036593,42.855629,44.036593,42.855629
0004d182d9fede3ba2534b2d5e5ad27e,43.585000,39.723000,43.586274,39.724272
0008c2445518c9392cb356c5c3db3392,51.528755,46.040150,51.529419,46.038521
000b373cc4969c0be8e0933c08da67e1,56.247688,43.463735,56.232037,43.458107
000c589e94c95984721de4b2bfb9ee4e,54.496312,37.062297,54.506758,37.066789
001611e3ac051a0ec91c88bbd9dbeb5a,56.999000,40.960000,56.963687,40.988294
001691ae3885e80add35148a01e75206,59.966977,30.310077,60.010055,30.261636
002044159304738ea7e3598131809851,53.570000,49.253000,53.518092,49.271210


In [66]:
preds2.fillna(0).to_csv("submission3.csv")

In [40]:
pd.read_csv("submission3.csv")

,_ID_,_WORK_LAT_,WORK_LON_,_HOME_LAT_,HOME_LON_
0,00021683ccb416637fe9a4cd35e4606e,55.023355,82.914724,55.038211,82.977366
1,0002d0f8a642272b41c292c12ab6e602,44.031000,42.838000,44.036593,42.855629
2,0004d182d9fede3ba2534b2d5e5ad27e,43.585000,39.723000,43.586274,39.724272
3,0008c2445518c9392cb356c5c3db3392,51.528755,46.040150,51.533936,46.025489
4,000b373cc4969c0be8e0933c08da67e1,56.232037,43.458107,56.232037,43.458107
5,000c589e94c95984721de4b2bfb9ee4e,54.496312,37.062297,54.506758,37.066789
6,001611e3ac051a0ec91c88bbd9dbeb5a,56.999000,40.960000,56.963687,40.988294
7,001691ae3885e80add35148a01e75206,59.966977,30.310077,60.010055,30.261636
8,002044159304738ea7e3598131809851,53.570000,49.253000,53.518092,49.271210
9,0026319faa345a573522f0a04f5c55bb,54.647549,39.647358,54.647549,39.647358


In [ ]:
folds_generator = GroupShuffleSplit(1,random_state=42)
ind = folds_generator.split(train_df, train_df.y_home, groups=train_df.customer_id)
folds = []
for el, er in ind:
    folds += [(el, er)]

In [ ]:
def fit(X_train, y_train, cols, num_boost_round=101, mt='xgb', typ="home"):
    if typ=="home":
        good_merch = X_train.home_add_lat.dropna().index
        X_train = X_train.loc[good_merch]
        y_train = y_train.loc[good_merch]
    else:
        good_merch = X_train.work_add_lat.dropna().index
        X_train = X_train.loc[good_merch]
        y_train = y_train.loc[good_merch]
    if mt == 'xgb':
        dtrain = xgb.DMatrix(X_train[cols], y_train)
        params = {'eta': 0.1,
          'objective': 'binary:logistic',
          'eval_metric': ['auc'],
          'max_depth': 3}
        model = xgb.train(params, dtrain, num_boost_round=num_boost_round, verbose_eval=10)
    if mt == 'lgb':
        lgb_train = lgb.Dataset(X_train[cols], y_train)

        # specify your configurations as a dict
        params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': 'binary_logloss',
            'bagging_freq': 0,
            'colsample_bytree': '0.514',
            'learning_rate': '0.063',
            'feature_fraction': '0.625',
            'bagging_fraction': '0.844',
            'num_leaves': 60,
            'verbose': 1
        }

        #print('Start training...')
        # train
        model = lgb.train(params,
                lgb_train,
                num_boost_round=100)

    return model

In [ ]:
def predict(model, X_test, cols, mt):
    if mt == 'xgb': proba = model.predict(xgb.DMatrix(X_test[cols]))
    if mt == 'lgb': proba = model.predict(X_test[cols], num_iteration=model.best_iteration)
    X_test['proba'] = proba
    mapping = X_test.groupby("customer_id").proba.max()
    X_test["max_proba"] = X_test.customer_id.map(mapping)
    X_test = X_test.loc[X_test.proba==X_test.max_proba,]
    lat = X_test.groupby("customer_id").pos_address_lat.median()
    lon = X_test.groupby("customer_id").pos_address_lon.median()
    return pd.merge(lat.to_frame("lat"), lon.to_frame("lon"), right_index=True, left_index=True)

In [ ]:
def evaluate(df, preds, typ):
    df = df[df.customer_id.isin(preds.index)]
    df = df[["customer_id",typ+"_add_lat", typ+"_add_lon"]].drop_duplicates()#
    customers = df.customer_id.value_counts()
    customers = customers[customers==1].index
    df = df[df.customer_id.isin(customers)]
    df = df.set_index("customer_id")
    res = np.array(df.loc[preds.index]) - np.array(preds)
    r = np.sum(np.sqrt(res[:, 0]**2+res[:, 1]**2) < .02)/res.shape[0]

    return r

In [ ]:
cols = set(list(train_df.columns)) - set(['cust_id_upd', 'city_upd', 'currency_upd', 'country_upd', 'mean_target_cutomer', 'mean_target_terminal', 'mean_target_city', 'mean_target_country',  'near_ter', 'coord_prob', 'lon_prob', 'lat_prob', 'amount_mean','week_day', 'pos', 'pos_address','mcc', 'atm_address', 'atm_address_lat', 'atm_address_lon', 'city',
       'country', 'currency', 'customer_id', 'home_add_lat', 'home_add_lon','terminal_id', 'transaction_date', 'work_add_lat', 'work_add_lon', 'y_home', 'y_work'])
cols = list(cols)

In [ ]:
model = fit(train_df.iloc[folds[0][0]], train_df.y_home.iloc[folds[0][0]], cols, typ="home", mt='lgb')

In [ ]:
preds = predict(model, train_df.iloc[folds[0][1], :].copy(), cols, mt='lgb')
evaluate(train_df.copy(), preds, "home")

In [ ]:
train_df[cols]

In [ ]:
model = fit(train_df.iloc[folds[0][0]], train_df.y_work.iloc[folds[0][0]], cols, typ="work", mt='lgb')

In [ ]:
preds = predict(model, train_df.iloc[folds[0][1], :].copy(), cols, mt='lgb')
evaluate(train.copy(), preds, "work")

In [ ]:
good_merch = temp.home_add_lat.dropna().index
X_train = temp.loc[good_merch]
y_train = temp.y_home.loc[good_merch]

lgb_train = lgb.Dataset(X_train[cols], y_train)
lgb_eval = lgb.Dataset(temp_test[cols], temp_test.y_home, reference=lgb_train)

        # specify your configurations as a dict
params = {
            'boosting_type': 'gbdt',
            'objective': 'binary',
            'metric': 'binary_logloss',
            'bagging_freq': 1,
            'colsample_bytree': '0.514',
            'learning_rate': '0.063',
            'feature_fraction': '0.625',
            'bagging_fraction': '0.844',
            'num_leaves': 60,
            'verbose': 1
            #'is_unbalance':True
        }

        #print('Start training...')
        # train
model = lgb.train(params,
                lgb_train,
                  valid_sets=lgb_eval,
                  verbose_eval=100,
                  early_stopping_rounds=10,
                num_boost_round=3000)


In [95]:
from hyperopt import hp, tpe
from hyperopt.fmin import fmin

In [ ]:
def objective(params):
    params = {
        'boosting_type': 'gbdt',  
        'objective': 'binary',
        'metric': 'binary_logloss',
        'bagging_freq': int(params['bagging_freq']),
        'colsample_bytree': '{:.3f}'.format(params['colsample_bytree']),
        'learning_rate': '{:.3f}'.format(params['learning_rate']),
        'feature_fraction': '{:.3f}'.format(params['feature_fraction']),
        'bagging_fraction': '{:.3f}'.format(params['bagging_fraction']),
        'num_leaves': int(params['num_leaves']),
    }
    X_train, y_train = train_df.iloc[folds[0][0]], train_df.y_home.iloc[folds[0][0]]
    good_merch = X_train.home_add_lat.dropna().index
    X_train = X_train.loc[good_merch]
    y_train = y_train.loc[good_merch]
    
    lgb_train = lgb.Dataset(X_train[cols], y_train)

    
    model = lgb.train(params,
                lgb_train,
                num_boost_round=100)
    
    preds = predict(model, train_df.iloc[folds[0][1], :].copy(), cols, mt='lgb')
    score = evaluate(train.copy(), preds, "home")
    print("Score {:.3f} params {}".format(score, params))
    return 1-score

space = {
    'num_leaves': hp.quniform('num_leaves', 8, 128, 2),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.3, 1.0),
    'bagging_freq': hp.quniform('bagging_freq', 1, 30, 3),
    'learning_rate': hp.uniform('learning_rate', 0.005, 0.5),
    'feature_fraction': hp.uniform('feature_fraction', 0.6, 0.95),
    'bagging_fraction': hp.uniform('bagging_fraction', 0.6, 0.95)}

best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=1000)

In [ ]:
best